<a href="https://colab.research.google.com/github/SebastianLarssonDTU/02456-Reinforcement-Learning-Project/blob/remove_fixed_values/getting_started_ppo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

import imageio

from datetime import datetime
from pytz import timezone 

import glob

# Getting started with PPO and ProcGen

Here's a bit of code that should help you get started on your projects.

The cell below installs `procgen` and downloads a small `utils.py` script that contains some utility functions. You may want to inspect the file for more details.

In [ ]:
!pip install procgen
!wget https://raw.githubusercontent.com/nicklashansen/ppo-procgen-utils/main/utils.py

# Mounting Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#check folders/files in drive
!ls drive/'My Drive'/'02456-Deep-Learning-Project'
data_path = 'drive/My Drive/02456-Deep-Learning-Project/Data/'

In [ ]:
def test_ability_to_create_file_on_drive():
  #Check we can create a log file
  columns = ['Step', 'Mean reward']
  dummy_df = pd.DataFrame(np.random.randn(100, 2), columns=columns)
  dummy_df
  #Using , as seperator so I could open in google sheets and verify data
  dummy_df.to_csv(data_path+'/dummy_test.csv', index=None, sep=',', mode='w')
  
  f = open(data_path+'/dummy_test.csv', "a")
  f.write("\n I can also append")
  f.close()

#test_ability_to_create_file_on_drive()

# Helper functions

In [ ]:
#Copied from https://lab-ml.com/labml_nn/rl/ppo/

def ClippedPPOLoss(log_pi: torch.Tensor, sampled_log_pi: torch.Tensor, advantage: torch.Tensor, clip: float) -> torch.Tensor:
  ratio = torch.exp(log_pi - sampled_log_pi)
  clipped_ratio = ratio.clamp(min=1.0 - clip, max=1.0 + clip)
  policy_reward = torch.min(ratio * advantage, clipped_ratio * advantage)
  #clip_fraction = (abs((ratio - 1.0)) > clip).to(torch.float).mean()
  return -policy_reward.mean()

def ClippedValueFunctionLoss(value: torch.Tensor, sampled_value: torch.Tensor, sampled_return: torch.Tensor, clip: float):
  clipped_value = sampled_value + (value - sampled_value).clamp(min=-clip, max=clip)
  vf_loss = torch.max((value - sampled_return) ** 2, (clipped_value - sampled_return) ** 2)
  return 0.5 * vf_loss.mean()

In [ ]:
def extract_data_from_csv(file_name):
  df = pd.read_csv(data_path+file_name)
  return df

In [ ]:
def create_index_table_from_txt_files():
  all_txt_files = glob.glob(data_path +'*.txt')
  # all_dfs=[]
  final_df = pd.DataFrame()
  
  for file in all_txt_files:
    df=pd.read_csv(file)
    df = df.set_index('Parameter name')
    df = df.transpose()
    # all_dfs.append(df)
    final_df = final_df.append(df)
  # return all_dfs
  return final_df

def update_index_file_with_result():
  pass

In [ ]:
create_index_table_from_txt_files()

#Baseline Hyper Params
Hyperparameters. These values should be a good starting point. You can modify them later once you have a working implementation.

In [ ]:
# Hyperparameters
total_steps = 8e6
num_envs = 32
num_levels = 10
num_steps = 256
num_epochs = 3
batch_size = 512
eps = .2
grad_eps = .5
value_coef = .5
entropy_coef = .01


feature_dim= 512    # <- The only thing we chose ourself

#Fixed values
in_channels = 3 #RGB
num_actions = 15  #Number of actions in the Procgen environment


# Network definitions 
We have defined a policy network for you in advance. It uses the popular `NatureDQN` encoder architecture (see below), while policy and value functions are linear projections from the encodings. There is plenty of opportunity to experiment with architectures, so feel free to do that! Perhaps implement the `Impala` encoder from [this paper](https://arxiv.org/pdf/1802.01561.pdf) (perhaps minus the LSTM).

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from utils import make_env, Storage, orthogonal_init


class Flatten(nn.Module):
    def forward(self, x):
        return x.view(x.size(0), -1)


class Encoder(nn.Module):
  def __init__(self, in_channels, feature_dim):
    super().__init__()
    self.layers = nn.Sequential(
        nn.Conv2d(in_channels=in_channels, out_channels=32, kernel_size=8, stride=4), 
        nn.ReLU(),
        nn.Conv2d(in_channels=32, out_channels=64, kernel_size=4, stride=2), 
        nn.ReLU(),
        nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1), 
        nn.ReLU(),
        Flatten(),
        nn.Linear(in_features=1024, out_features=feature_dim), 
        nn.ReLU()
    )
    self.apply(orthogonal_init)

  def forward(self, x):
    return self.layers(x)


class Policy(nn.Module):
  def __init__(self, encoder, feature_dim, num_actions):
    super().__init__()
    self.encoder = encoder
    self.policy = orthogonal_init(nn.Linear(feature_dim, num_actions), gain=.01)
    self.value = orthogonal_init(nn.Linear(feature_dim, 1), gain=1.)

  def act(self, x):
    with torch.no_grad():
      x = x.cuda().contiguous()
      dist, value = self.forward(x)
      action = dist.sample()
      log_prob = dist.log_prob(action)
    
    return action.cpu(), log_prob.cpu(), value.cpu()

  def forward(self, x):
    x = self.encoder(x)
    logits = self.policy(x)
    value = self.value(x).squeeze(1)
    #Creates a categorical distribution parameterized by either probs or logits
    dist = torch.distributions.Categorical(logits=logits)
    #sample with dist.sample()
    return dist, value

# Training definition

In [ ]:
"""
  TODO: 
    1. Maybe update this to take hyperparams as input 
    2. Update to do data logging in files instead of as prints
"""
def train_network(print_output=False, 
                  file_name=None,
                  total_steps = 8e6,
                  num_envs = 32,
                  num_levels = 10,
                  num_steps = 256,
                  num_epochs = 3,
                  batch_size = 512,
                  eps = .2,
                  grad_eps = .5,
                  value_coef = .5,
                  entropy_coef = .01,
                  lr=5e-4,
                  opt_extra = 1e-5,
                  gamma=0.99,
                  lmbda = 0.95):
  
  if file_name is None:
    now = datetime.now(timezone('Europe/Copenhagen'))
    file_name = 'Training_Run_' + now.strftime("%d%b_%Hh%Mm%Ss")
    
  hyper_params={}
  
  for key, val in reversed(list(locals().items())):
    if key in ['print_output', 'hyper_params', 'now']:
      continue
    hyper_params[key] = val


  
  #create file
  f = open(data_path+file_name+'.csv', "w")
  f.write("Step, Mean reward")
  f.close()

  #create txt file for hyper params
  f = open(data_path+file_name+'.txt', "w")
  f.write("Parameter name, Value")
  for key, val in hyper_params.items():  
    f.write("\n{}, {}".format(key, val))
  f.close()

  # Define environment
  # check the utils.py file for info on arguments
  env = make_env(num_envs, num_levels=num_levels)
  if print_output:
    print('Observation space:', env.observation_space)
    print('Action space:', env.action_space.n)

  encoder = Encoder(in_channels = in_channels, feature_dim = feature_dim)
  policy = Policy(encoder = encoder, feature_dim = feature_dim, num_actions = num_actions)
  policy.cuda()

  # Define optimizer
  # these are reasonable values but probably not optimal
  optimizer = torch.optim.Adam(policy.parameters(), lr=lr, eps=opt_extra)

  # Define temporary storage
  # we use this to collect transitions during each iteration
  storage = Storage(
      env.observation_space.shape,
      num_steps,
      num_envs,
      gamma = gamma,
      lmbda = lmbda
  )

  # Run training
  obs = env.reset()
  step = 0
  while step < total_steps:

    # Use policy to collect data for num_steps steps
    policy.eval()
    for _ in range(num_steps):
      # Use policy
      action, log_prob, value = policy.act(obs)
      
      # Take step in environment
      next_obs, reward, done, info = env.step(action)

      # Store data
      storage.store(obs, action, reward, done, info, log_prob, value)
      
      # Update current observation
      obs = next_obs

    # Add the last observation to collected data
    _, _, value = policy.act(obs)
    storage.store_last(obs, value)

    # Compute return and advantage
    storage.compute_return_advantage()

    # Optimize policy
    policy.train()
    for epoch in range(num_epochs):

      # Iterate over batches of transitions
      generator = storage.get_generator(batch_size)
      for batch in generator:
        b_obs, b_action, b_log_prob, b_value, b_returns, b_advantage = batch

        # Get current policy outputs
        new_dist, new_value = policy(b_obs)
        new_log_prob = new_dist.log_prob(b_action)

        # Clipped policy objective
        #Assume pi_loss = ClippedPPOLoss
        pi_loss = ClippedPPOLoss(log_pi= new_log_prob, 
                                sampled_log_pi=b_log_prob,
                                advantage=b_advantage, 
                                clip=eps)

        # Clipped value function objective
        #Assume value_loss = ClippedValueFunctionLoss 
        value_loss = ClippedValueFunctionLoss(value= new_value, 
                                              sampled_value=b_value, 
                                              sampled_return= b_returns, #prob not right!
                                              clip=eps)

        # Entropy loss
        entropy_loss = new_dist.entropy().mean()

        # Backpropagate losses
        loss = pi_loss + value_coef * value_loss - entropy_coef*entropy_loss
        loss.backward()

        # Clip gradients
        torch.nn.utils.clip_grad_norm_(policy.parameters(), grad_eps)

        # Update policy
        optimizer.step()
        optimizer.zero_grad()

    # Update stats
    step += num_envs * num_steps
    if print_output:
      print(f'Step: {step}\tMean reward: {storage.get_reward()}')
    f = open(data_path+file_name+'.csv', "a")
    f.write("\n{}, {}".format(step, storage.get_reward()))
    f.close()

  if print_output:
    print('Completed training!')
  torch.save(policy.state_dict, 'checkpoint.pt')
  

#Do Training

In [ ]:
train_network(print_output=True)

# Post training processing 
Below cell can be used for policy evaluation and saves an episode to mp4 for you to view.

In [ ]:
def policy_evaluation(video_name='vid', print_output=False):
  # Make evaluation environment
  eval_env = make_env(num_envs, start_level=num_levels, num_levels=num_levels)
  obs = eval_env.reset()

  frames = []
  total_reward = []

  # Evaluate policy
  policy.eval()
  for _ in range(512):

    # Use policy
    action, log_prob, value = policy.act(obs)

    # Take step in environment
    obs, reward, done, info = eval_env.step(action)
    total_reward.append(torch.Tensor(reward))

    # Render environment and store
    frame = (torch.Tensor(eval_env.render(mode='rgb_array'))*255.).byte()
    frames.append(frame)

  # Calculate average return
  total_reward = torch.stack(total_reward).sum(0).mean(0)
  if print_output:
    print('Average return:', total_reward)

  # Save frames as video
  frames = torch.stack(frames)
  imageio.mimsave(video_name+'.mp4', frames, fps=25)
  
  return total_reward

# Grid search ?


In [ ]:
# Hyperparameters
total_steps = [8e6]
num_envs = [32]
num_levels = [10]
num_steps = [256]
num_epochs = [3]
batch_size = [8,512]
eps = [.2]
grad_eps = [.5]
value_coef = [.5]
entropy_coef = [.01]

for total_steps, num_envs, ..., in list:
  train()


feature_dim= 512    # <- The only thing we chose ourself

#Fixed values
in_channels = 3 #RGB
num_actions = 15  #Number of actions in the Procgen environment